In [42]:
from __future__ import print_function
from googleapiclient.discovery import build
import gspread
from httplib2 import Http
from oauth2client import file, client, tools
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
from itertools import islice
import re
from lxml import etree


# Key takeaways

* Use British-English right from the beginning
* Force the people to be responsible for their stuff
* Force the people to work on mendeley
* Force the people to use harvard citation
* Force the people to write texts with the roughly the same length

# How does this script work?!

* The order of the trends is done by the order of the trends_intro and then by the inner order of the groups

# Configuration

## Headlines

In [6]:
# Abbreviations-Section
ABBREVIATION_TITLE = "List of Abbrevations"
ABBREVIATION_HEADLINE_TAG = "H6"

# Trends-Section
TRENDS_TITLE = "Trends"
TRENDS_DESCRIPTION = ""
TRENDS_SUB_SECTION_HEADLINE_TAG = "H2"
TRENDS_SUB_SECTION_SLOGAN_TAG = "H3"
TRENDS_SUB_SECTION_AREA_HEADLINE_TAG = "H4" # Trend Drivers, Trend Facts ... 
TRENDS_SUB_SECTION_AREA_BULLET_ICON = "•" # List icon for Trend Drivers, Trend Facts ...
TRENDS_SUB_SECTION_AREA_IMPACT_HEADLINE = "Impact on XXX"

# Sources-Section
SOURCES_TITLE = "Sources"
SOURCES_DESCRIPTION = ""
SOURCES_KEY_TAG = "H6"

# Pipeline

This pipeline downloads the data from the googel spread sheet and then replaced

# 1. Help functions

## 1.1. Text formatation

In [7]:
def sanitize_text(text):
    text = text.replace("&", "&amp;")
    return text

In [8]:
def sanitize_text_test():
    print(sanitize_text("Hello & World") == "Hello &amp; World")

#sanitize_text_test()
    

In [9]:
def generate_xml_list(text):
    result = "<List>\n"
    li = [s.strip() for s in text.splitlines()]
    for l in li:
        result += "<List-Element>" + TRENDS_SUB_SECTION_AREA_BULLET_ICON + " " + l+ "</List-Element>\n"

    result += "</List>"
    return result

In [10]:
def generate_xml_list_test():
    print(generate_xml_list("Hello World \n hello / Seb \n hello CDTM\n"))
    
#generate_xml_list_test()

## 1.2. Find and replace author

In [11]:
def find_author_and_replace(text):    
    counter = 0
    authors = dict()
    p = re.compile("\[([a-zäöüÄÖÜA-Z_0-9]*)")
    for match in p.finditer(text):
        key = match.group(1)
        if key not in authors and len(key) > 0:
            counter += 1
            authors[key] = counter
    
    # print(authors)
            
    for key, value in authors.items():
        text = text.replace("["+str(key), "["+str(value))
        
    return (text, authors)

In [12]:
def find_author_and_replace_test():
    test = "The side bar include [] [KRAUZ, p.22] a Cheatsheet, full [KRAUZ] Reference, sults with the Tools below [SEB], [KAYA]. Replace & List outp [HASE]. [KRAUZ] [KRAUZ] [KRAUZ] [KRAUZ]"
    text, authors = find_author_and_replace(test)
    print(text)
    print(authors)

    
find_author_and_replace_test()

The side bar include [] [1, p.22] a Cheatsheet, full [1] Reference, sults with the Tools below [2], [3]. Replace & List outp [4]. [1] [1] [1] [1]
{'KRAUZ': 1, 'HASE': 4, 'SEB': 2, 'KAYA': 3}


## Download the data from the spreadsheet

http://www.countingcalculi.com/explanations/google_sheets_and_jupyter_notebooks/

In [13]:
# If modifying these scopes, delete the file token.json.
SCOPES = 'https://www.googleapis.com/auth/spreadsheets.readonly'

# The ID and range of a sample spreadsheet.
SPREADSHEET_ID = '16jza4slLRK4Fe3-O_f410P_-WZBe5EN-fmq_OW3HJNQ'
RANGE_NAME = 'Trends'

In [14]:
def load_data_from_google_sheets():
    scope = ['https://www.googleapis.com/auth/spreadsheets.readonly']
    credentials = ServiceAccountCredentials.from_json_keyfile_name('./seb_credentials.json', scope)
    print(credentials)
    gc = gspread.authorize(credentials)
    
    book = gc.open_by_key(SPREADSHEET_ID)
    return book

In [15]:
book = load_data_from_google_sheets();
print(book)

<Spreadsheet u'TSF2018_TrendSeminar' id:16jza4slLRK4Fe3-O_f410P_-WZBe5EN-fmq_OW3HJNQ>


# XML

## Scenarios

Generate scenario XML from excel sheet
```
<scenarios>
  <scenario>
    <title>
    <H1>shitty title</H1>
    </title>
    
    <subtitle>
    <H2>shitty subtitle</H2>
    </subtitle>
    
    <text>
    sometext
    </text>
  </scenario>
  <scenario>
      .
      .
      .
  </scenario>
  ...
  ...
  ...
  <scenario>
      .
      .
      .
  </scenario>
</scenarios>
```

In [52]:

def listify_sign_posts(text):
    li = [s.strip() for s in text.splitlines()]
    root = etree.Element("sign_posts")
    for item in li:
        if item == "" or item == "\n":
            continue
        post = etree.Element("sign_post")
        post.text = item
        root.append(post)
    return root
        
def generate_scenario_xml(book):
    scenariosheet = book.worksheet("Scenarios")
    table = scenariosheet.get_all_values()
    df_scenarios = pd.DataFrame(table[2:], columns=table[0])
    
    # Build the XML tree
    root = etree.Element("scenarios")
    
    for index, row in islice(df_scenarios.iterrows(), 0, None):
        scenario = etree.Element("scenario")
        
        title = etree.Element("title")
        h1 = etree.Element("h1")
        h1.text = sanitize_text(row[2])
        title.append(h1)
        scenario.append(title)
        
        subtitle = etree.Element("subtitle")
        h2 = etree.Element("h2")
        h2.text =  sanitize_text(row[5])
        subtitle.append(h2)
        scenario.append(subtitle)
        
        text = etree.Element("text")
        text.text = sanitize_text(row[7])
        scenario.append(text)
        
        #sign_posts = etree.Element("sign_posts")
        #sign_posts.text = generate_xml_list(sanitize_text(row[9]))
        scenario.append(listify_sign_posts(sanitize_text(row[9])))
        
        root.append(scenario)
        
    return etree.tostring(root).decode('utf-8')
        
book = load_data_from_google_sheets() 
xml = generate_scenario_xml(book)
print(xml)

<scenarios><scenario><title><h1>Butler industry</h1></title><subtitle><h2>A day in 2050</h2></subtitle><text>A modern cottage in the Allg&#228;u region with a wooden terrace, a friendly garden, and a spacious garage for their self-driving car - this morning Christine wakes up with a big smile on her face. This is what she and her spouse have always dreamed of and as of today, it will become reality. She gets dressed full of excitement, puts on her augmented reality glasses, and grabs a quick smoothie, before she leaves for the high-speed capsule train from Munich to Allg&#228;u. When the government started to build this new network of high-speed capsule trains, Christine and her husband immediately bought a lovely property in the countryside. Like all of her friends, Christine follows the Build it Yourself (BIY) trend and wants to have a unique home with individual style. Through 3D printing technologies, customized houses are on the rise. For example, a befriended couple just built a 

## Ideas

Generate Ideas XML from excel sheet
```
<ideas>
    <idea>
        <title>
            <h1>
            </h1>
        </title>
        <subtitle>
            <h2>
            </h2>
        </subtitle>
        <Value_Proposition_Canvas>
            <item>
            </item>
            <item>
            </item>
        </Value_Proposition_Canvas>
        <Value_Proposition_Text>
            <text>
            </text>
        </Value_Proposition_Text>
        ...
        ...
        ...
    </idea>
    ...
    ...
</ideas>
```  

In [51]:
def listify_canvas(text, root_tag):
    li = [s.strip() for s in text.splitlines()]
    root = etree.Element(root_tag)
    for item in li:
        if item == "" or item == "\n":
            continue
        post = etree.Element("item")
        post.text = item
        root.append(post)
    return root

def generate_ideas_xml(book):
    ideasheet = book.worksheet("Ideation")
    table = ideasheet.get_all_values()
    df_ideas = pd.DataFrame(table[3:])#, columns=table[3])
    #print(df_ideas.head)
    #df_ideas.set_index('Title',inplace=True)
    df_ideas = df_ideas.transpose()
    df_ideas = df_ideas[1:]
    print(df_ideas.shape)#['Title'])
    # Build the XML tree
    ideas = etree.Element("ideas")
    
    for index, row in islice(df_ideas.iterrows(), 0, None):
        if (row[0]) == "":
            continue
        idea = etree.Element("idea")
        
        # Title
        title = etree.Element("title")
        heading1 = etree.Element("h1")
        heading1.text = row[0]
        title.append(heading1)
        idea.append(title)
        
        #subtitle
        sub = etree.Element("subtitle")
        h2 = etree.Element("h2")
        h2.text = row[1]
        sub.append(h2)
        idea.append(sub)
        
        #intro
        intro = etree.Element("intro")
        text = etree.Element("text")
        text.text = row[2]
        intro.append(text)
        idea.append(intro)
        
        #Value Proposition_Canvas
        #vpc = etree.Element("Value_Proposition_Canvas")
        #text = etree.Element("text")
        #text.text = row[4]
        #vpc.append(listify_canvas(row[4], "Value_Proposition_Canvas"))
        idea.append(listify_canvas(row[4], "Value_Proposition_Canvas"))
        
        #Value Proposition_Text
        vpt = etree.Element("Value_Proposition_Text")
        text = etree.Element("text")
        text.text = row[5]
        vpt.append(text)
        idea.append(vpt)        
        
        #Customer Relationships_Canvas
        #crc = etree.Element("Customer_Relationships_Canvas")
        #text = etree.Element("text")
        #text.text = row[6]
        #crc.append(text)
        idea.append(listify_canvas(row[6], "Customer_Relationships_Canvas"))  
        
        #Customer Relationships_Text
        crt = etree.Element("Customer_Relationships_Text")
        text = etree.Element("text")
        text.text = row[7]
        crt.append(text)
        idea.append(crt)  
        
        #Channels_Canvas
        #cc = etree.Element("Channels_Canvas")
        #text = etree.Element("text")
        #text.text = row[8]
        #cc.append(text)
        idea.append(listify_canvas(row[8], "Channels_Canvas"))
        
        #Channels_Text
        ct = etree.Element("Channels_Text")
        text = etree.Element("text")
        text.text = row[9]
        ct.append(text)
        idea.append(ct)
        
        #Key Resources_Canvas
        #krc = etree.Element("Key_Resources_Canvas")
        #text = etree.Element("text")
        #text.text = row[10]
        #krc.append(text)
        idea.append(listify_canvas(row[10], "Key_Resources_Canvas"))
        
        #Key Resources_Text
        krt = etree.Element("Key_Resources_Text")
        text = etree.Element("text")
        text.text = row[11]
        krt.append(text)
        idea.append(krt)
        
        #Key Activities_Canvas
        #kac = etree.Element("Key_Activities_Canvas")
        #text = etree.Element("text")
        #text.text = row[12]
        #kac.append(text)
        idea.append(listify_canvas(row[12], "Key_Activities_Canvas"))
        
        #Key Activities_Text
        kat = etree.Element("Key_Activities_Text")
        text = etree.Element("text")
        text.text = row[13]
        kat.append(text)
        idea.append(kat)
        
        #Revenue Streams_Canvas
        #rsc = etree.Element("Revenue_Streams_Canvas")
        #text = etree.Element("text")
        #text.text = row[14]
        #rsc.append(text)
        idea.append(listify_canvas(row[14], "Revenue_Streams_Canvas"))
        
        #Revenue Streams_Text
        rst = etree.Element("Revenue_Streams_Text")
        text = etree.Element("text")
        text.text = row[15]
        rst.append(text)
        idea.append(rst)
        
        #Key Partners_Canvas
        #kpc = etree.Element("Key_Partners_Canvas")
        #text = etree.Element("text")
        #text.text = row[16]
        #kpc.append(text)
        idea.append(listify_canvas(row[16], "Key_Partners_Canvas"))
        
        #Key Partners_Text
        kpt = etree.Element("Key_Partners_Text")
        text = etree.Element("text")
        text.text = row[17]
        kpt.append(text)
        idea.append(kpt)
        
        #Customer Segmentation_Canvas
        #csc = etree.Element("Customer_Segmentation_Canvas")
        #text = etree.Element("text")
        #text.text = row[18]
        #csc.append(text)
        idea.append(listify_canvas(row[18], "Customer_Segmentation_Canvas"))
        
        #Customer Segmentation_Text
        cst = etree.Element("Customer_Segmentation_Text")
        text = etree.Element("text")
        text.text = row[19]
        cst.append(text)
        idea.append(cst)
        
        #Cost Structure_Canvas
        #csc = etree.Element("Cost Structure_Canvas")
        #text = etree.Element("text")
        #text.text = row[20]
        #csc.append(text)
        idea.append(listify_canvas(row[20], "Cost_Structure_Canvas"))
        
        #Cost Structure_Text
        cst = etree.Element("Cost_Structure_Text")
        text = etree.Element("text")
        text.text = row[21]
        cst.append(text)
        idea.append(cst)
        
        #Senario Fit_1
        sf = etree.Element("Senario_Fit_1")
        text = etree.Element("text")
        text.text = row[23]
        sf.append(text)
        idea.append(sf)
        
        #Senario Fit_2
        sf = etree.Element("Senario_Fit_2")
        text = etree.Element("text")
        text.text = row[24]
        sf.append(text)
        idea.append(sf)
        
        #Senario Fit_3
        sf = etree.Element("Senario_Fit_3")
        text = etree.Element("text")
        text.text = row[25]
        sf.append(text)
        idea.append(sf)
        
        #Senario Fit_4
        sf = etree.Element("Senario_Fit_4")
        text = etree.Element("text")
        text.text = row[26]
        sf.append(text)
        idea.append(sf)
        
        
        
        ideas.append(idea)
        print(row[0])
        
    return etree.tostring(ideas).decode('utf-8')
book = load_data_from_google_sheets() 
xml = generate_ideas_xml(book)
print(xml)

(6, 30)
Conas
Craidar
<ideas><idea><title><H1>Conas</H1></title><subtitle><H2>A Virtual Audio Assistant for On-Site Efficiency</H2></subtitle><intro><text>Conas is a cloud-based audio assistance designed for on-site construction workers. With Conas, they are supported in extracting information from complex construction plans and hence, in delivering their assigned tasks efficiently and according to requirements. Construction plans are usually developed as 3D models in architectural software and &#8211; in a steadily increasing number of cases &#8211; implemented in BIM. However, on construction sites, workers still rely on 2D paper-based plans as a source of measurements, material information, and interior object location. Conas provides a digitized solution in translating the information of a construction plan into audio. &#13;
From a technical perspective, this solution is enabled by the Conas design-to-speech software. Architects and construction companies upload their BIM and CAD d

## Abbrevations

Generate the abbrevations from the sheet and transform it into the XML structure

```
<Abbreviations>
<Abbreviation><H6>AI</H6>Artificial Intelligence</Abbreviation>
<Abbreviation><H6>BIM</H6>Building information modeling</Abbreviation>
<Abbreviation><H6>LCC</H6>Life Cycle Costing</Abbreviation>
...
</Abbreviations>
```

In [ ]:
def generate_abbrevations(book):
    worksheet = book.worksheet("Abreviations")
    table = worksheet.get_all_values()
    ##Convert table data into a dataframe
    df = pd.DataFrame(table[1:], columns=table[0])
    # Init XML structure
    result = "<Abbreviations-Section>\n";
    result += "<H1>"+ABBREVIATION_TITLE+"</H1>\n";
    result += "<Abbreviations>\n";
    # Iterate over the rows
    for index, row in islice(df.iterrows(), 1, None):
        # Grab the key and values
        key = sanitize_text(row[0])
        val = sanitize_text(row[1])
        # Create a xml string
        result += "<Abbreviation><"+ABBREVIATION_HEADLINE_TAG+">" + key + "</"+ABBREVIATION_HEADLINE_TAG+">" + val + "</Abbreviation>\n"
    # Add closing tag
    result += "</Abbreviations>\n";
    result += "</Abbreviations-Section>\n";
    return result

In [ ]:
print(generate_abbrevations(book))

## Trends

Generate the trends from the sheet and transform it into the XML structure

```
<Trends-Section>
<List>
<List-Element>Technology Trends</List-Element>
<List-Element>Societal &amp; Environmental Trends</List-Element>
</List>
<Trends-Sub-Sections>
<Trends-Sub-Section>
<H1>AI</H1>
<H3>SUBRTITLE</H3>
<Text>Intro text for bla bla</Text>
<Trends>
<Trend>
<H2>Trend 1 Title</H2>
<H3>Slogan</H3>
<Text>
Intro text
<H4>Facts</H4>
<List>
<List-Element>Hello World</List-Element>
<List-Element>Hello CDTM!</List-Element>
<List-Element>Hello Sebastian</List-Element>
...
</List>
</Text>
</Trend>
</Trends>
</Trends-Sub-Section>
...
</Trends-Sub-Sections>


<Abbreviation><H6>BIM</H6>Building information modeling</Abbreviation>
<Abbreviation><H6>LCC</H6>Life Cycle Costing</Abbreviation>
...
</Abbreviations>
```

In [ ]:
def generate_trends(book):
    # Init XML structure
    result = "<Trends-Section>\n";
    result += "<H1>"+TRENDS_TITLE+"</H1>\n";
    # Add description if necessary
    if len(TRENDS_DESCRIPTION) > 0:
        result += "<Text>"+TRENDS_DESCRIPTION+"</Text>\n";
    
    # Init list of trend sections
    result_trend_list = "<List>\n"
    
    # Load trends
    # Trends_intro
    worksheet = book.worksheet("Trend_Intro")
    table = worksheet.get_all_values()
    ##Convert table data into a dataframe
    df_trends_intro = pd.DataFrame(table[1:], columns=table[0])
    # print(df_trends_intro)

    worksheet = book.worksheet("Trends")
    table = worksheet.get_all_values()
    # Convert table data into a dataframe
    df_trends = pd.DataFrame(table[2:], columns=table[0])
    # print(df_trends)
    
    # Start with sub sections
    result_sub_sections = "<Trends-Sub-Sections>\n"
    # Iterate over the trend intro
    for index, row in islice(df_trends_intro.iterrows(), 0, None):
        # Grab the key and values
        key = sanitize_text(row[3])
        intro_text = sanitize_text(row[4])
        intro_responsible = sanitize_text(row[2])
        
        # Init the trend sub section
        result_trend_sub_section = '<Trends-Sub-Section title="'+key+'">\n';
        
        result_trend_sub_section += "<H1>"+ key + "</H1>\n"
        result_trend_sub_section += '<Text responsible="'+intro_responsible+'">'+ intro_text + "</Text>\n"
        
        # Add the trend to the overview list
        result_trend_list += "<List-Element>" + key + "</List-Element>\n"
        
        # Start adding the trends
        result_trend_sub_section += '<Trends>\n'
        
        for trend_index, trend_row in df_trends.loc[df_trends['Sub-Section'] == row[3]].iterrows():
            trend_title = sanitize_text(trend_row[2])
            trend_slogan = sanitize_text(trend_row[7])
            trend_intro = sanitize_text(trend_row[9])
            trend_facts = sanitize_text(trend_row[11])
            trend_drivers = sanitize_text(trend_row[13])
            trend_challanges = sanitize_text(trend_row[15])
            trend_impact = sanitize_text(trend_row[17])
            trend_responsible = sanitize_text(trend_row[5])
            
            result_trend = '<Trend responsible="'+trend_responsible+'">\n'            
            
            result_trend += "<"+TRENDS_SUB_SECTION_HEADLINE_TAG+">" + trend_title + "</"+TRENDS_SUB_SECTION_HEADLINE_TAG+">\n"
            result_trend += "<"+TRENDS_SUB_SECTION_SLOGAN_TAG+">" + trend_slogan + "</"+TRENDS_SUB_SECTION_SLOGAN_TAG+">\n"
            result_trend += "<Text>\n"
            # Trend intro
            result_trend += trend_intro + "\n"
            # Trend Facts
            result_trend += "<"+TRENDS_SUB_SECTION_AREA_HEADLINE_TAG+">"+"Facts:"+"</"+TRENDS_SUB_SECTION_AREA_HEADLINE_TAG+">"+"\n"
            result_trend += generate_xml_list(trend_facts) +"\n"
            # Trend Key Drivers
            result_trend += "<"+TRENDS_SUB_SECTION_AREA_HEADLINE_TAG+">"+"Key Drivers:"+"</"+TRENDS_SUB_SECTION_AREA_HEADLINE_TAG+">"+"\n"
            result_trend += generate_xml_list(trend_drivers) +"\n"
            # Trend Challenges
            result_trend += "<"+TRENDS_SUB_SECTION_AREA_HEADLINE_TAG+">"+"Challenges:"+"</"+TRENDS_SUB_SECTION_AREA_HEADLINE_TAG+">"+"\n"
            result_trend += generate_xml_list(trend_challanges) +"\n"
            # Trend Impact Headline
            result_trend += "<"+TRENDS_SUB_SECTION_AREA_HEADLINE_TAG+">"+TRENDS_SUB_SECTION_AREA_IMPACT_HEADLINE +":"+"</"+TRENDS_SUB_SECTION_AREA_HEADLINE_TAG+">"+"\n"
            result_trend += generate_xml_list(trend_impact) +"\n"
            # Trend Impact Text
            result_trend += "</Text>\n"
            result_trend += "</Trend>\n"
            result_trend_sub_section += result_trend
        
        # Close the trend section
        result_trend_sub_section += '</Trends>\n'
    
        
    
        # Close the trend sub section
        result_trend_sub_section += "</Trends-Sub-Section>\n";

        # Add it to the result
        result_sub_sections += result_trend_sub_section;

    result_sub_sections += "</Trends-Sub-Sections>"
    
    result_trend_list += "</List>"
    
    # Add the elements to the result object
    result += result_trend_list + "\n"
    result += result_sub_sections + "\n"
    result += "</Trends-Section>\n";
    return result  

## Sources

Generate the sources from the sheet and transform it into the XML structure.
Order the sources by the apperance.

```
<Sources-Section>
<H1>Source</H1>
<Sources>
<Source>
<H6>1</H6> Sebastians Report 2017 ....
</Source>
...
</Sources>
</Sources-Section>
```

In [ ]:
def generate_sources(book, map_hash_source):
    # invert the mapping NUMBER -> HASH
    map_number_hash = {v: k for k, v in map_hash_source.items()}
    
    # Init an array with the size of the sources
    sources = [("","")]*len(map_hash_source)
       
    worksheet = book.worksheet("Sources")
    table = worksheet.get_all_values()
    # Convert table data into a dataframe
    df = pd.DataFrame(table[1:], columns=table[0])
    # print(df_trends)
    
    # Start with sub sections
    result = "<Sources-Sections>\n"
    result += "<H1>"+SOURCES_TITLE+"</H1>\n"
    result += "<Text>"+SOURCES_DESCRIPTION+"</Text>\n"
    # Start with the list
    result += "<Sources>\n"
    # Iterate over the sources array
    for index, row in islice(df.iterrows(), 1, None):
        key = sanitize_text(row[0])
        responsible = sanitize_text(row[2])
        val = sanitize_text(row[4])
        if key in map_hash_source:
            # Find the number of the source
            source_index = map_hash_source[key]-1
            # Add the value of the source to the right order of the array
            sources[source_index] = (val, responsible)
        else:
            #print("Source not used", responsible , key)
        
    # Iterate of the ordered source array and genrate the xml
    for index, (source, responsible) in enumerate(sources):
        
        if (len(source) == 0):
            #print("Source not declared", map_number_hash[index+1])
        else:
            result += '<Source responsible="'+responsible+'">\n'
            result += "<" + SOURCES_KEY_TAG + ">" + str(index+1) + "</" + SOURCES_KEY_TAG + "> "
            result += source
            result += "\n"
            result += "</Source>\n"
    
    # Close the xml tags
    result += "</Sources>\n"    
    result += "</Sources-Sections>\n"
    
    return result

In [ ]:
def run():
    book = load_data_from_google_sheets()
    xml = ""
    # Add Abbreviations
    xml += generate_abbrevations(book)
    # Add Trends
    xml += generate_trends(book)
    # Add Scenarios
    # Add Ideation
    # Add Sources    


    # Replace the citation
    xml, authors = find_author_and_replace(xml)
    print(authors)
    xml += generate_sources(book, authors)
    # Wrap the root object arround all
    xml = '<?xml version="1.0" encoding="UTF-8" standalone="yes"?>\n<Root>\n' + xml + '</Root>'
    #print(xml)
    return xml

In [ ]:
run()